In [33]:
from arcexplore import *
import pandas as pd
from dataclasses import dataclass

from typing import Callable

In [6]:
ARC_Sets = {}
for key in ARC_DATA:
    ARC_Sets[f"{key} Training"] = ARC_DATA[key].training
    ARC_Sets[f"{key} Evaluation"] = ARC_DATA[key].evaluation



In [41]:
@dataclass
class ArcAnalysis:
    data: Mapping[str, Mapping[str, Mapping[str, Task]]]
    cols: tuple
    analyze: Callable

    def tabulate(self):
        rows = [(k,) + self.analyze(v) for k,v in self.data.items()]
        columns = ("Dataset",) + self.cols
        return pd.DataFrame.from_records(rows, columns=columns, index="Dataset")

In [49]:
def count_tasks(dset: Mapping[str,Task]):
    grid_count = [len(d.test)+len(d.train) for _, d in dset.items()]
    sum_points = lambda x: sum([np.prod(y.input.shape) + np.prod(y.output.shape) for y in x])
    grid_points = [sum_points(d.test)+sum_points(d.train) for _, d in dset.items()]
    return (len(dset), sum(grid_count), sum(grid_points))

ArcAnalysis(data=ARC_Sets,
            cols=("Tasks","Grid Pairs", "Grid Points"),
            analyze=count_tasks).tabulate()

,Tasks,Grid Pairs,Grid Points
Dataset,,,
ARC-AGI-1 Training,400,1718,443235
ARC-AGI-1 Evaluation,400,1782,742351
ARC-AGI-2 Training,1000,4308,1443669
ARC-AGI-2 Evaluation,120,529,384199


In [52]:
def count_equal_tasks(dset: Mapping[str,Task]):
    eq = 0
    check_eq = lambda t: 1 if tuple(t.input.shape) == tuple(t.output.shape) else 0
    for _, task in dset.items():
        ind_eq = sum([check_eq(t) for t in task.train])
        ind_eq += sum([check_eq(t) for t in task.test])
        if ind_eq == (len(task.train) + len(task.train)):
            eq += 1
    return (eq, len(dset))

ArcAnalysis(data=ARC_Sets,
            cols=("Equal Tasks","Total Tasks"),
            analyze=count_equal_tasks).tabulate()

,Equal Tasks,Total Tasks
Dataset,,
ARC-AGI-1 Training,1,400
ARC-AGI-1 Evaluation,0,400
ARC-AGI-2 Training,6,1000
ARC-AGI-2 Evaluation,12,120
